In [1]:
!pip install python-pptx
!pip install openai

In [2]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

import tkinter as tk
from tkinter import filedialog
from pptx import Presentation

# Connect to OpenAI (use API KEY)

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.

## Define class PowerPoint to crawl contents from chosen `.pptx` file
- Use `tkinter` to display dialog that allows user choose .pptx file
- Use `Presentation` to extract contents from .pptx file
  

In [5]:
class PowerPoint:
    def __init__(self):
        root = tk.Tk()
        root.withdraw()  # Hide the root window
        self.file_path = filedialog.askopenfilename(title="Select a PowerPoint File",
                                               filetypes=[("PowerPoint Files", "*.pptx")])
        self.title = None
        self.text = None
        if self.file_path:
            self.text = self._extract_text_from_pptx()
            self.title = self._extract_title_from_pptx()

    def _extract_text_from_pptx(self):
        prs = Presentation(self.file_path)
        text_content = []
    
        for slide_number, slide in enumerate(prs.slides, start=1):
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    slide_text.append(shape.text)
            text_content.append(f"Slide {slide_number}:\n" + "\n".join(slide_text))
    
        return "\n\n".join(text_content)
    
    def _extract_title_from_pptx(self):
        prs = Presentation(self.file_path)
    
        # Attempt to extract from document properties (metadata)
        if prs.core_properties.title:
            return prs.core_properties.title
    
        # Fallback: Try to get the first slide's title
        if prs.slides:
            for shape in prs.slides[0].shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    return shape.text.strip()
    
        return "No Title Found"
        

In [6]:
presentation = PowerPoint()
print(presentation.text)

2025-03-02 13:35:24.281 python[22732:454667] +[CATransaction synchronize] called within transaction


Slide 1:
Chapter 2:  Operating-System Services

Slide 2:
Outline
Operating System Services
User and Operating System-Interface
System Calls
System Services
Linkers and Loaders
Why Applications are Operating System Specific
Design and Implementation
Operating System Structure
Building and Booting an Operating System
Operating System Debugging




Slide 3:
Objectives
Identify services provided by an operating system
Illustrate how system calls are used to provide operating system services
Compare and contrast monolithic, layered, microkernel, modular, and hybrid strategies for designing operating systems
Illustrate the process for booting an operating system
Apply tools for monitoring operating system performance
Design and implement kernel modules for interacting with a Linux kernel


Slide 4:
Operating System Services
Operating systems provide an environment for execution of programs and services to programs and users
One set of operating-system services provides functions that are hel

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt 
system_prompt = "You are an assistant that analyzes the contents of a pptx file \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown in Vietnamese."

In [8]:
# A function that writes a User Prompt that asks for summaries of a presentation:
def user_prompt_for(presentation):
    user_prompt = f"You are looking at a presentation titled {presentation.title}"
    user_prompt += "\nThe contents of this presentation is as follows; \
please provide a short summary of this presentation in markdown. \
If it includes news, announcements or references, then summarize these too.\n\n"
    user_prompt += presentation.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [9]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

## And now let's build useful messages for GPT-4o-mini, using a function

In [10]:
# See how this function creates exactly the format above

def messages_for(presentation):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(presentation)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [11]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize():
    presentation = PowerPoint()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(presentation)
    )
    return response.choices[0].message.content

In [12]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary():
    summary = summarize()
    display(Markdown(summary))

In [13]:
display_summary()

2025-03-02 13:35:26.287 python[22732:454667] +[CATransaction synchronize] called within transaction


# Tóm tắt chương 2: Các dịch vụ của hệ điều hành

Chương 2 của bài thuyết trình tập trung vào các dịch vụ do hệ điều hành cung cấp, các interfaces giữa người dùng và hệ điều hành, cũng như các cuộc gọi hệ thống và dịch vụ hệ thống. Dưới đây là những điểm chính trong nội dung:

## Các dịch vụ của hệ điều hành
- Hệ điều hành cung cấp môi trường để thực thi chương trình và dịch vụ cho người dùng.
- Các dịch vụ gồm:
  - **Giao diện người dùng**: CLI, GUI, cảm ứng.
  - **Chạy chương trình**: Tải và thực hiện chương trình.
  - **Hoạt động I/O**: Quản lý dữ liệu và thiết bị ngoại vi.
  - **Quản lý hệ thống tệp**: Thao tác với tệp và thư mục.

## Gọi hệ thống
- Cung cấp giao diện lập trình cho các dịch vụ của hệ điều hành.
- Hệ thống gọi thường được truy cập qua các API như Win32, POSIX và Java API.

## Các loại gọi hệ thống
- **Quản lý tiến trình**: Tạo, kết thúc và quản lý các đặc điểm của tiến trình.
- **Quản lý tệp**: Thao tác với các tệp (tạo, xóa, đọc, ghi).
- **Quản lý thiết bị**: Yêu cầu và giải phóng các thiết bị.
- **Thông tin**: Lấy và thiết lập thời gian, dữ liệu hệ thống.
- **Giao tiếp**: Gửi, nhận thông điệp giữa các tiến trình.

## Thiết kế và triển khai
- Chiến lược thiết kế hệ điều hành có thể bao gồm cấu trúc đơn nhất, phân lớp, vi kernel, và hybrid.
- Quan điểm về chính sách và cơ chế cần được tách biệt để linh hoạt hơn trong việc thay đổi chính sách.

## Xây dựng và khởi động hệ điều hành
- Quy trình xây dựng hệ điều hành bao gồm viết mã nguồn, cấu hình, biên dịch và cài đặt.
- Cần có mã khởi động để tải và khởi động hệ điều hành khi bật máy.

## Gỡ lỗi và tối ưu hóa hiệu suất
- Gỡ lỗi liên quan đến việc phát hiện và sửa lỗi, cũng như tối ưu hóa hiệu suất hệ thống.
- Các công cụ gỡ lỗi và theo dõi như `strace`, `gdb`, và `perf` được sử dụng để thu thập dữ liệu cho các sự kiện cụ thể.

Bài thuyết trình này cung cấp cái nhìn sâu sắc về chức năng và cấu trúc của các dịch vụ trong hệ điều hành, cũng như cách thức các ứng dụng tương tác với hệ thống.